In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.arima.model import ARIMA# ar and ima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller#time series is stationary or not
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
deliveries=pd.read_csv(r"C:\Users\shata\OneDrive\Desktop\puthon\deliveries.csv")

In [3]:
#Handling duplicates in team
team_dup={"Delhi Daredevils":"Delhi Capitals",
"Rising Pune Supergiants":"Rising Pune Supergiant",
"Royal Challengers Bengaluru":"Royal Challengers Bangalore",'Gujarat Lions':'Gujarat Titans'}
deliveries.replace(to_replace=team_dup,inplace=True)

In [4]:
vkohli=deliveries[deliveries.batter=="V Kohli"]
vkohli_score=vkohli.groupby('match_id')['batsman_runs'].sum().reset_index()
vkohli_score_sub=vkohli_score.head(239)
vkohli_score.tail(5)

,match_id,batsman_runs
239,1426290,42
240,1426296,92
241,1426300,27
242,1426306,47
243,1426310,33


In [5]:
S_Dhawan=deliveries[deliveries.batter=="S Dhawan"]
S_Dhawan_score=S_Dhawan.groupby('match_id')['batsman_runs'].sum().reset_index()
S_Dhawan_score_sub=S_Dhawan_score.head(239)
print(S_Dhawan_score)
S_Dhawan_score.tail(5)

     match_id  batsman_runs
0      335984            52
1      335988            25
2      335995             0
3      335998            50
4      336001            19
..        ...           ...
216   1422120            22
217   1422124            45
218   1422129            70
219   1422135             1
220   1426261            14

[221 rows x 2 columns]


,match_id,batsman_runs
216,1422120,22
217,1422124,45
218,1422129,70
219,1422135,1
220,1426261,14


In [6]:
batter=deliveries.groupby(['match_id','batter']).agg({'batsman_runs':'sum'}).reset_index()
batter_scores=batter.groupby('batter')['batsman_runs'].sum().reset_index()
batter_scores=batter_scores.sort_values(by="batsman_runs",ascending=False)
top_batter_scores=batter_scores.head(10)
print(top_batter_scores)
for i in range(len(top_batter_scores)):
  batsman=top_batter_scores.iloc[i]["batter"]
  print(batsman)
  score=batter[batter['batter']==batsman]
  print(score)

             batter  batsman_runs
631         V Kohli          8014
512        S Dhawan          6769
477       RG Sharma          6630
147       DA Warner          6567
546        SK Raina          5536
374        MS Dhoni          5243
30   AB de Villiers          5181
124        CH Gayle          4997
501      RV Uthappa          4954
282      KD Karthik          4843
V Kohli
       match_id   batter  batsman_runs
13       335982  V Kohli             1
56       335985  V Kohli            23
168      335992  V Kohli            13
224      335996  V Kohli            12
252      335998  V Kohli             1
...         ...      ...           ...
16197   1426290  V Kohli            42
16300   1426296  V Kohli            92
16367   1426300  V Kohli            27
16434   1426306  V Kohli            47
16479   1426310  V Kohli            33

[244 rows x 3 columns]
S Dhawan
       match_id    batter  batsman_runs
35       335984  S Dhawan            52
96       335988  S Dhawan            

In [7]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Example data (replace with your actual data)
# top_batter_scores and batter should be defined here


In [8]:
print(top_batter_scores.head())
print(batter.head())


        batter  batsman_runs
631    V Kohli          8014
512   S Dhawan          6769
477  RG Sharma          6630
147  DA Warner          6567
546   SK Raina          5536
   match_id       batter  batsman_runs
0    335982    AA Noffke             9
1    335982      B Akhil             0
2    335982  BB McCullum           158
3    335982     CL White             6
4    335982    DJ Hussey            12


In [9]:
for i in range(len(top_batter_scores)):
    batsman = top_batter_scores.iloc[i]["batter"]
    print(f"Batsman: {batsman}")

    # Filter and prepare data
    score = batter[batter['batter'] == batsman]
    print(score.head())  # Verify the filtered data
    break  # Temporarily exit loop to test one iteration


Batsman: V Kohli
     match_id   batter  batsman_runs
13     335982  V Kohli             1
56     335985  V Kohli            23
168    335992  V Kohli            13
224    335996  V Kohli            12
252    335998  V Kohli             1


In [10]:
score["match_number"] = range(len(score))
score.set_index("match_number", inplace=True)

# Splitting into train and test sets
train_size = int(len(score) * 0.8)
train, test = score["batsman_runs"][:train_size], score["batsman_runs"][train_size:]

print(f"Train Data:\n{train.head()}")
print(f"Test Data:\n{test.head()}")


Train Data:
match_number
0     1
1    23
2    13
3    12
4     1
Name: batsman_runs, dtype: int64
Test Data:
match_number
195    25
196    51
197    53
198    39
199    41
Name: batsman_runs, dtype: int64


C:\Users\shata\AppData\Local\Temp\ipykernel_11380\1244873902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score["match_number"] = range(len(score))


In [11]:
model = SARIMAX(
    train,
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1, 12)
)
model_fit = model.fit(disp=False)
print(model_fit.summary())


                                     SARIMAX Results                                      
Dep. Variable:                       batsman_runs   No. Observations:                  195
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 12)   Log Likelihood                -867.964
Date:                            Wed, 01 Jan 2025   AIC                           1745.928
Time:                                    20:58:33   BIC                           1761.948
Sample:                                         0   HQIC                          1752.422
                                            - 195                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0473      0.082      0.573      0.567      -0.114       0.209
ma.L1         -0.9241      0.034   

C:\Users\shata\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [12]:
# Forecasting for test set
test_forecast = model_fit.forecast(steps=len(test))

# Forecasting beyond the dataset
future_forecast = model_fit.forecast(steps=5)

# Evaluation metrics
mae = mean_absolute_error(test, test_forecast)
mse = mean_squared_error(test, test_forecast)
rmse = np.sqrt(mse)

# Display results
print("Future Forecasted Values (Next 5 Matches):")
print(future_forecast)
print("Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


Future Forecasted Values (Next 5 Matches):
195    20.366466
196    20.113327
197    21.847781
198    21.352350
199    44.513481
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 26.29
Mean Squared Error (MSE): 1087.48
Root Mean Squared Error (RMSE): 32.98


In [13]:
try:
    # Code block
except Exception as e:
    print(f"Error with batsman {batsman}: {e}")


IndentationError: expected an indented block after 'try' statement on line 1 (2791871761.py, line 3)

In [14]:
bowler_data = deliveries[['match_id', 'bowler', 'is_wicket', 'total_runs']]


performance = bowler_data.groupby(['match_id', 'bowler']).agg(
    wickets=('is_wicket', 'sum'),
    runs_conceded=('total_runs', 'sum')
).reset_index()

print(performance)

top_bowler = 'YS Chahal'
bowler_timeseries = performance[performance['bowler'] == top_bowler].set_index('match_id')['wickets']
print(bowler_timeseries)

bowler_timeseries = bowler_timeseries.sort_index()

       match_id         bowler  wickets  runs_conceded
0        335982      AA Noffke        1             41
1        335982     AB Agarkar        3             25
2        335982       AB Dinda        2              9
3        335982       CL White        0             24
4        335982       I Sharma        1             13
...         ...            ...      ...            ...
12973   1426312     PJ Cummins        1             18
12974   1426312      SP Narine        1             20
12975   1426312  Shahbaz Ahmed        1             28
12976   1426312    T Natarajan        0             29
12977   1426312       VG Arora        1             26

[12978 rows x 4 columns]
match_id
598029     0
729281     1
729287     2
729299     1
729305     1
          ..
1426294    1
1426299    1
1426303    2
1426310    1
1426311    0
Name: wickets, Length: 159, dtype: int64


In [15]:
bowlers=performance.groupby('bowler')['wickets'].sum().reset_index()
top_bowl=bowlers.sort_values(by="wickets",ascending=False)
top_bowlers=top_bowl.head(10)
top_bowlers

,bowler,wickets
524,YS Chahal,213
119,DJ Bravo,207
348,PP Chawla,201
446,SP Narine,200
355,R Ashwin,198
71,B Kumar,195
438,SL Malinga,188
8,A Mishra,183
193,JJ Bumrah,182
373,RA Jadeja,169


In [16]:
for i in range(len(top_bowlers)):
  bowler=top_bowlers.iloc[i]["bowler"]
  print(bowler)
  bscore=performance[performance['bowler']==bowler]
  print(bscore)

YS Chahal
       match_id     bowler  wickets  runs_conceded
4236     598029  YS Chahal        0             35
4782     729281  YS Chahal        1             18
4816     729287  YS Chahal        2             17
4889     729299  YS Chahal        1             26
4923     729305  YS Chahal        1             17
...         ...        ...      ...            ...
12793   1426294  YS Chahal        1             48
12851   1426299  YS Chahal        1             22
12890   1426303  YS Chahal        2             31
12952   1426310  YS Chahal        1             43
12965   1426311  YS Chahal        0             34

[159 rows x 4 columns]
DJ Bravo
       match_id    bowler  wickets  runs_conceded
84       335989  DJ Bravo        0             37
110      335991  DJ Bravo        0             27
146      335994  DJ Bravo        0             31
182      335997  DJ Bravo        1             25
267      336004  DJ Bravo        1             29
...         ...       ...      ...           

In [17]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Example datasets (replace with actual data)
# top_bowlers and performance should already be defined
print(top_bowlers.head())  # Check top bowlers data
print(performance.head())  # Check performance data


        bowler  wickets
524  YS Chahal      213
119   DJ Bravo      207
348  PP Chawla      201
446  SP Narine      200
355   R Ashwin      198
   match_id      bowler  wickets  runs_conceded
0    335982   AA Noffke        1             41
1    335982  AB Agarkar        3             25
2    335982    AB Dinda        2              9
3    335982    CL White        0             24
4    335982    I Sharma        1             13


In [18]:
# Select the first bowler
bowler = top_bowlers.iloc[0]["bowler"]
print(f"Selected Bowler: {bowler}")

# Filter performance data
bscore = performance[performance['bowler'] == bowler]

# Add match numbers and set index
bscore["match_number"] = range(len(bscore))
bscore.set_index("match_number", inplace=True)

print(bscore.head())  # Check prepared data


Selected Bowler: YS Chahal
              match_id     bowler  wickets  runs_conceded
match_number                                             
0               598029  YS Chahal        0             35
1               729281  YS Chahal        1             18
2               729287  YS Chahal        2             17
3               729299  YS Chahal        1             26
4               729305  YS Chahal        1             17


C:\Users\shata\AppData\Local\Temp\ipykernel_11380\1800594547.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bscore["match_number"] = range(len(bscore))


In [19]:
# Splitting data
train_size = int(len(bscore) * 0.8)  # 80% train, 20% test
train, test = bscore["wickets"][:train_size], bscore["wickets"][train_size:]

# Print split details
print(f"Train Data:\n{train.head()}")
print(f"Test Data:\n{test.head()}")


Train Data:
match_number
0    0
1    1
2    2
3    1
4    1
Name: wickets, dtype: int64
Test Data:
match_number
127    0
128    0
129    1
130    4
131    1
Name: wickets, dtype: int64


In [20]:
# Fit SARIMA model
model = SARIMAX(train, order=(2, 1, 2), seasonal_order=(2, 1, 2, 12))
model_fit = model.fit()

# Print model summary
print(model_fit.summary())


                                     SARIMAX Results                                      
Dep. Variable:                            wickets   No. Observations:                  127
Model:             SARIMAX(2, 1, 2)x(2, 1, 2, 12)   Log Likelihood                -182.058
Date:                            Wed, 01 Jan 2025   AIC                            382.117
Time:                                    21:02:56   BIC                            406.743
Sample:                                         0   HQIC                           392.111
                                            - 127                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.7387      0.209     -3.535      0.000      -1.148      -0.329
ar.L2         -0.0765      0.153   

In [21]:
# Forecast for test set
test_forecast = model_fit.forecast(steps=len(test))

# Forecast beyond the dataset (e.g., next 5 matches)
future_forecast = model_fit.forecast(steps=5)

# Evaluation metrics
mae = mean_absolute_error(test, test_forecast)
mse = mean_squared_error(test, test_forecast)
rmse = np.sqrt(mse)

# Print results
print("Future Forecasted Values (Next 5 Matches):")
print(future_forecast)
print("Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


Future Forecasted Values (Next 5 Matches):
127    1.616748
128    2.548456
129    1.681790
130    2.791391
131    1.648513
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 1.14
Mean Squared Error (MSE): 1.87
Root Mean Squared Error (RMSE): 1.37


In [22]:
for i in range(len(top_bowlers)):
    bowler = top_bowlers.iloc[i]["bowler"]
    print(f"Bowler: {bowler}")

    # Filter and prepare data
    bscore = performance[performance['bowler'] == bowler]
    bscore["match_number"] = range(len(bscore))
    bscore.set_index("match_number", inplace=True)

    # Split data
    train_size = int(len(bscore) * 0.8)
    train, test = bscore["wickets"][:train_size], bscore["wickets"][train_size:]

    # Fit and forecast
    try:
        model = SARIMAX(train, order=(2, 1, 2), seasonal_order=(2, 1, 2, 12))
        model_fit = model.fit()
        test_forecast = model_fit.forecast(steps=len(test))
        future_forecast = model_fit.forecast(steps=5)

        # Evaluation metrics
        mae = mean_absolute_error(test, test_forecast)
        mse = mean_squared_error(test, test_forecast)
        rmse = np.sqrt(mse)

        # Print results
        print("Future Forecasted Values (Next 5 Matches):")
        print(future_forecast)
        print("Evaluation Metrics:")
        print(f"Mean Absolute Error (MAE): {mae:.2f}")
        print(f"Mean Squared Error (MSE): {mse:.2f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    except Exception as e:
        print(f"Error processing bowler {bowler}: {e}")

    print("-" * 50)


Bowler: YS Chahal


C:\Users\shata\AppData\Local\Temp\ipykernel_11380\230031394.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bscore["match_number"] = range(len(bscore))


Future Forecasted Values (Next 5 Matches):
127    1.616748
128    2.548456
129    1.681790
130    2.791391
131    1.648513
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 1.14
Mean Squared Error (MSE): 1.87
Root Mean Squared Error (RMSE): 1.37
--------------------------------------------------
Bowler: DJ Bravo


C:\Users\shata\AppData\Local\Temp\ipykernel_11380\230031394.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bscore["match_number"] = range(len(bscore))
C:\Users\shata\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\shata\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


Future Forecasted Values (Next 5 Matches):
126    0.807416
127    0.260067
128    0.632067
129    1.181742
130    1.383186
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 0.93
Mean Squared Error (MSE): 1.34
Root Mean Squared Error (RMSE): 1.16
--------------------------------------------------
Bowler: PP Chawla


...

Future Forecasted Values (Next 5 Matches):
140    1.728953
141    0.728069
142    0.913187
143    0.995499
144    1.241284
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 0.85
Mean Squared Error (MSE): 1.63
Root Mean Squared Error (RMSE): 1.28
--------------------------------------------------
Bowler: SL Malinga


C:\Users\shata\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\shata\AppData\Local\Temp\ipykernel_11380\230031394.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bscore["match_number"] = range(len(bscore))


Future Forecasted Values (Next 5 Matches):
97     2.060859
98     1.215058
99     0.444244
100    1.315642
101    1.417425
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 1.18
Mean Squared Error (MSE): 2.13
Root Mean Squared Error (RMSE): 1.46
--------------------------------------------------
Bowler: A Mishra
Future Forecasted Values (Next 5 Matches):
129    0.465799
130    0.342719
131    0.412837
132    0.892323
133    0.558713
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 0.92
Mean Squared Error (MSE): 1.32
Root Mean Squared Error (RMSE): 1.15
--------------------------------------------------
Bowler: JJ Bumrah


C:\Users\shata\AppData\Local\Temp\ipykernel_11380\230031394.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bscore["match_number"] = range(len(bscore))
C:\Users\shata\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
C:\Users\shata\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\shata\AppData\Local\Temp\ipykernel_11380\230031394.py:7: SettingW

Future Forecasted Values (Next 5 Matches):
106    2.508118
107    2.234863
108    1.184167
109    1.849315
110    3.875173
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 1.77
Mean Squared Error (MSE): 4.02
Root Mean Squared Error (RMSE): 2.00
--------------------------------------------------
Bowler: RA Jadeja
Future Forecasted Values (Next 5 Matches):
168    0.566482
169    0.871262
170    0.351423
171    1.461107
172    0.572671
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 0.79
Mean Squared Error (MSE): 0.97
Root Mean Squared Error (RMSE): 0.98
--------------------------------------------------
